In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
# import required libraries
!pip install pymongo
!pip install xgboost
import pandas as pd
import json
# import pymongo which creates the connection between python and mongoDB
import pymongo
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# creating a client for pymongo
client = pymongo.MongoClient("mongodb://localhost:27017")
# importing our CSV file into a pandas dataframe
df = pd.read_csv("C:\data\db\colon.csv")

df=df.dropna()

# converting the dataframe into dictionary (JSON like structure)
data = df.to_dict(orient="records")

# creating a new database "db"
db = client["colon"]

# inserting the dictionary into collection "colonCancer" in the database "db"
db.colonCancer.insert_many(data)

# creating a client for pymongo
client = pymongo.MongoClient("mongodb://localhost:27017")
# creating collection object to read "colonCancer" collection from database "db"
myCollection = db.get_collection("colonCancer")
# reading all documents at a time
# it returns a iterable cursor
allRecords = myCollection.find()
# converting the cursor into a list
# list is easy to operate
listCursor = list(allRecords)
# converting the list into pandas dataframe
df = pd.DataFrame(listCursor)

for i in range(100):
    # getting independent variables
    X=df[['rx','sex','age','obstruct','perfor','adhere','nodes','differ','extent','surg','time']]
    y=df[['status']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    rfr = RandomForestRegressor(n_estimators = 100)
    knn = KNeighborsRegressor(n_neighbors=20, metric='euclidean')
    ada = AdaBoostRegressor()
    #ada.fit(X_train,y_train)
    xgb = XGBRegressor(max_depth=3, learning_rate=0.02, n_estimators=200, min_child_weight=5, gamma=0.4, colsample_bytree=0.6, subsample=0.8, reg_alpha=1.1)
    xgb.fit(X_train,y_train)
    #y_pred = ada.predict(X_test)
    y_pred = xgb.predict(X_test)
    y_test = np.asarray(y_test)
    def perf_measure(y_actual, y_hat):
        TP = 0
        FP = 0
        TN = 0
        FN = 0
        for i in range(len(y_hat)): 
            if y_actual[i]==y_hat[i]==1:
                TP += 1
            if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
                FP += 1
            if y_actual[i]==y_hat[i]==0:
                TN += 1
            if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
                FN += 1
        return(TP, FP, TN, FN)
    a=np.where(y_test<0.5,0,1)
    b=np.where(y_pred<0.5,0,1)
    tp,fp,tn,fn=perf_measure(a,b)
    acc=(tp+tn)/(tp+fp+tn+fn)
    print("Accuracy: :",acc)
    f = open(r"C:\Users\Ripan\Desktop\accuXGB_PM.txt","a+")
    acc=str(format(acc, '.4f'))
    acc=acc+'\n'
    f.writelines(acc)
    f.close()

Accuracy: : 0.9550561797752809
Accuracy: : 0.9550561797752809
Accuracy: : 0.9241573033707865
Accuracy: : 0.9325842696629213
Accuracy: : 0.9325842696629213
Accuracy: : 0.9325842696629213
Accuracy: : 0.9438202247191011
Accuracy: : 0.9550561797752809
Accuracy: : 0.949438202247191
Accuracy: : 0.9438202247191011
Accuracy: : 0.9438202247191011
Accuracy: : 0.9382022471910112
Accuracy: : 0.9213483146067416
Accuracy: : 0.949438202247191
Accuracy: : 0.9382022471910112
Accuracy: : 0.9325842696629213
Accuracy: : 0.9353932584269663
Accuracy: : 0.9241573033707865
Accuracy: : 0.9410112359550562
Accuracy: : 0.9269662921348315
Accuracy: : 0.9382022471910112
Accuracy: : 0.9297752808988764
Accuracy: : 0.9297752808988764
Accuracy: : 0.9269662921348315
Accuracy: : 0.9297752808988764
Accuracy: : 0.9410112359550562
Accuracy: : 0.9073033707865169
Accuracy: : 0.9353932584269663
Accuracy: : 0.9353932584269663
Accuracy: : 0.9410112359550562
Accuracy: : 0.949438202247191
Accuracy: : 0.952247191011236
Accuracy: : 